In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras import callbacks
import numpy as np
import pickle
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model, load_model
import jieba
import re, random, time

Using TensorFlow backend.


### Load encoder & decoder

In [2]:
encoder_model = load_model('../models/encoder_model.hdf5')
decoder_model = load_model('../models/decoder_model.hdf5')

F:\Anaconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
encoder_model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, None, 3125)        0         
_________________________________________________________________
encoder_lstm1 (LSTM)         [(None, None, 256), (None 3463168   
_________________________________________________________________
encoder_lstm2 (LSTM)         [(None, None, 256), (None 525312    
_________________________________________________________________
encoder_lstm3 (LSTM)         [(None, None, 256), (None 525312    
Total params: 4,513,792
Trainable params: 4,513,792
Non-trainable params: 0
_________________________________________________________________


In [4]:
decoder_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      (None, None, 3130)   0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           (None, 256)          0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 256)          0                                            
__________________________________________________________________________________________________
decoder_lstm1 (LSTM)            [(None, None, 256),  3468288     decoder_input[0][0]              
                                                                 input_13[0][0]             

### Load word2id, id2word

In [3]:
data_all = pickle.load(open("../models/data_word_id_map.pkl","rb"))
id2word, word2id = data_all[4], data_all[5]

In [4]:
# 规范化数据，转小写，去除标点等其他非字母符号
def filter_data(s):
    re_han = re.compile("([a-zA-Z]+)")
    s = s.lower()
    blocks = re_han.split(s)
    txt = ''
    for blk in blocks:
        if re_han.match(blk):
            txt = txt + blk + ' '
    return txt

In [5]:
def get_one_hot(label, wordNum):
    label_arr = [0] * wordNum
    label_arr[label]=1
    return label_arr[:]

In [6]:
def expendToOneHot(x, wordNum):
    t = np.expand_dims(x, axis=2)
    t = t.tolist()
    seqLen = x.shape[1]
    for i in range(len(t)):
        for j in range(seqLen):
            print(str(i)+'-'+str(j))
            if(t[i][j][0] == 0):
                t[i][j:] = [[0] * wordNum] * (seqLen - j)
                break
            else:
                t[i][j] = get_one_hot(t[i][j][0], wordNum)
    return np.array(t)

In [7]:
encoder_model.output

[<tf.Tensor 'encoder_lstm1/while:4' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'encoder_lstm1/while:5' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'encoder_lstm2/while:4' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'encoder_lstm2/while:5' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'encoder_lstm3/while:4' shape=(None, 256) dtype=float32>,
 <tf.Tensor 'encoder_lstm3/while:5' shape=(None, 256) dtype=float32>]

In [15]:
def inference(input_seq):
    # 将输入序列进行编码
    states_value = encoder_model.predict(input_seq)
    
    # happy
    emotion = [0,0,1,0,0]
    
    # 生成一个size=1的空序列
    target_seq = np.array([0]*3125 + emotion)
    target_seq = np.expand_dims(target_seq, axis=0)
    target_seq = np.expand_dims(target_seq, axis=0)
    
    # 将这个空序列的内容设置为开始字符
    target_seq[0, 0, word2id['<SOS>']] = 1
    
    # 进行字符恢复
    # 简单起见，假设batch_size = 1
    decoded_sentence = ''
    
    while True:
        output_tokens, state_h1, state_c1, state_h2, state_c2, state_h3, state_c3 = decoder_model.predict([target_seq] + states_value)

        # sample a token
        # 去掉最后5个情绪向量
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = id2word[sampled_token_index]

        # 退出条件：生成 <EOS> 或者 超过最大序列长度
        if sampled_char == '<EOS>' or len(decoded_sentence) > 50 :
            break

        decoded_sentence += sampled_char + " "

        # 更新target_seq
        x = np.array([0]*3125 + emotion)
        target_seq = np.expand_dims(target_seq, axis=0)
        target_seq = np.expand_dims(target_seq, axis=0)
        target_seq[0, 0, sampled_token_index] = 1.

        # 更新中间状态
        states_value = [state_h1, state_c1, state_h2, state_c2, state_h3, state_c3]
        
    return decoded_sentence

In [19]:
user_input = "hello"
# user_input = 'excuse me are you angry?'
# user_input = 'excuse me are you sad?'
user_input = filter_data(user_input).split()
user_input

['hello']

In [20]:
input_words = []
for i in user_input:
    input_words.append(word2id[i])
input_words = np.array(input_words)
input_words = np.expand_dims(input_words, axis=0)
input_words = expendToOneHot(input_words, 3125)

0-0


In [21]:
inference(input_words)

'juggle '